Importing Libraries
___

In [ ]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import math
plt.rcParams['figure.dpi'] = 1000 # Increase image resolution
plt.rcParams['savefig.dpi'] = 1000 # Increase image resolution
#import plotly.express as px

Importing, Visualising and Editing Battery Dataframe
___

In [ ]:
df = pd.read_csv('RW3together.csv') # Import Battery Dataset (RW3/RW4/RW5/RW6 = Battery 1/2/3/4 Respectively)
df.comment = pd.Categorical(df.comment) # Conversion to categorical variable

In [ ]:
print (df.comment.cat.categories) # Different Regimes present in the dataset

In [ ]:
# Selecting various regimes using conditions
#df.loc[(df.comment=='low current discharge at 0.04A') & (df.voltage ==3.2),:]
#df.loc[(df.comment=='pulsed load (discharge)'),:]

In [ ]:
# Dataset's Voltage Profile (vs Time) 
'''plt.figure(figsize=(12,4)) # Figsize altered to compare wrt the publication
plt.plot(df.time/3600,df.voltage,linewidth=0.75) # Line Plot of Voltage vs time
plt.ylabel('Voltage (V)',fontsize=15)
plt.xlabel('Time (h)',fontsize=15)
plt.xlim(68,74.5) # Manually define x-axis's limit to check different regimes
plt.ylim(3.1,4.3) # Voltage varies b/w 3.2 V -> 4.2 V
plt.show()
'''

In [ ]:
# Function to alter all the fontsizes(legend, axes, title, label) in plots/subplots etc.
# Using setfs(ax1,15) = All ax1 elements changed to fontsize 15 pt.
def setfs(ax, fs):
  vals = [ax.title, ax.xaxis.label,
  ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()
  if ax.get_legend() is not None:
    vals += ax.get_legend().get_texts()
  for item in vals:
      item.set_fontsize(fs)

In [ ]:
# Creating a Sub-Plot with Voltage, Current, Temperature and the Regime Profiles (vs Time) to track various regimes
plt.figure(figsize=(12,12)) # Setting 1:1 dimensions 
ax1 = plt.subplot(4,1,1) # Forming the first vertical subplot
ax1.plot(df.time/3600,df.voltage,linewidth=0.75) #Subplot for Voltage
ax2 = plt.subplot(4,1,2, sharex=ax1) # Forming the second vertical subplot
ax2.plot(df.time/3600,df.current,linewidth=0.75, color = "green") #Subplot for Current
ax3 = plt.subplot(4,1,3, sharex=ax1) # Forming the third vertical subplot
ax3.plot(df.time/3600,df.temperature,linewidth=0.75, color = "red") #Subplot for Temperature
ax1.set_ylabel('Voltage (V)',fontsize=20)
ax2.set_ylabel('Current (A)',fontsize=20)
ax3.set_ylabel('Temperature(C'+chr(176)+')',fontsize=20)
ax4 = plt.subplot(4,1,4, sharex=ax1) # Forming the fourth vertical subplot
ax4.plot(df.time/3600,df.comment,linewidth=0.75, color = "black") #Subplot for Regimes (Categorical)
# Using the above fontsize function
setfs(ax1,20)
setfs(ax2,20)
setfs(ax3,20)
setfs(ax4,20)
ax4.set_ylim(4.9,7.10) # Show Specific Regimes
ax1.set_xlim(75,225) # Sets all x-axis to given time limits
ax3.set_ylim(0,50) # Setting temperature in the required range (irregular temperature values in RW3 dataset)
plt.xlabel('Time (h)',fontsize=20) #Setting the X-axis label
plt.show()

In [ ]:
# Interactive graph to maximise/minimise zoom into different regimes 
'''from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=4, cols=1,shared_xaxes = True) # Forming a 4 row and 0 column (Stricly vertical) - Subplot
fig.append_trace(go.Line(x=df.time[0:600000],y=df.voltage[0:600000], name = "Voltage"), row=1, col=1) # First 
fig.append_trace(go.Line(x=df.time[0:600000],y=df.current[0:600000], name = "Current"), row=2, col=1) # Second 
fig.append_trace(go.Line(x=df.time[0:600000],y=df.temperature[0:600000], name = "Temperature"), row=3, col=1) # Third
fig.append_trace(go.Line(x=df.time[0:600000],y=df.comment[0:600000], name = "Mode"), row=4, col=1) # Fourth
# Setting Labels
fig.update_yaxes(title_text="Voltage(V)", row=1, col=1)
fig.update_yaxes(title_text="Current(A)", row=2, col=1)
fig.update_yaxes(title_text="Temperature(C"+chr(176)+")", range=[0, 50], row=3, col=1)
fig.update_yaxes(title_text="Regimes", row=4, col=1)
fig.update_xaxes(title_text="Time(h)")
fig.update_layout(autosize=False,width=1000,height=1000) # Setting 1:1 dimensions
fig.show()'''

In [ ]:
# Creating new dataframe (Section 3.3.1)
time1 = np.zeros(len(df)) # New time variable to perform dt calculations
time1[0] = df.time[0] # Initialising first value
for i in range(1,len(df)):
    time1[i] = df.time[i-1] # Shifting time variable by one value
dt = df.time - time1 # Subtracting the shifted gives the time difference (dt) between every datapoint 
df['dt'] = dt # Forming the new column
df1 = df[['comment','voltage','current','temperature','relativeTime','time','dt']] # Reordering the columns
dq = dt*df1["current"] # Forming a new variable with the charge difference (dq) between every datapoint
df1['dq'] = dq # Forming a new column ...

Modelling: Using Reference Paper -> Electrochemistry-based Battery Modeling for Prognostics
___

In [ ]:
#Constant Parameters in the Model: (Section 3.3.2)
start = 22768 # Index at the start of the first pulsed discharge regime
end = 45719 # Index at the end of the first pulsed discharge regime

# Table 3.1
Ap0 = -33642.23 # (J/mol)
Ap1 = 0.11 # (J/mol)
Ap2 = 23506.89 # (J/mol)
Ap3 = -74679.26 # (J/mol)
Ap4 = 14359.34 # (J/mol)
Ap5 = 307849.79 # (J/mol)
Ap6 = 85053.13 # (J/mol)
Ap7 = -1075148.06 # (J/mol)
Ap8 = 2173.62 # (J/mol)
Ap9 = 991586.68 # (J/mol)
Ap10 = 283423.47 # (J/mol)
Ap11 = -163020.34 # (J/mol)
Ap12 = -470297.35 # (J/mol)
Ap = [Ap0, Ap1, Ap2, Ap3, Ap4, Ap5, Ap6, Ap7, Ap8, Ap9, Ap10, Ap11, Ap12] # Forming vector with the simplex coefficients
An0 = 86.19 # (J/mol)
U0p = 4.03 # (V)
U0n = 0.01 # (V)

# Table 3.2
R = 8.314 # (J/mol/K)
T = 292 # (K)
F = 96487 # (C/mol)
n = 1 # (Dimensionless)
D = 7*(10**6) # (mol s/C/m^3)
tau0 = 10 # (s)
taup = 90 # (s)
taun = 90 # (s)
alpha = 0.5 # (Dimensionless)
R0 = 0.085 # (ohm)
QMAX = 1.32*(10**4) # (C)
Sp = 2*(10**-4) # (m^2)
Sn = 2*(10**-4) # (m^2)
kp = 2*(10**4) # (A/m^2)
kn = 2*(10**4) # (A/m^2)
vsp = 2*(10**-6) # (m^3)
vsn = 2*(10**-6) # (m^3)
vbp = 2*(10**-5) # (m^3)
vbn = 2*(10**-5) # (m^3)
vn = vsn + vbn # (m^3) (eq 3.9)
vp = vsp + vbp # (m^3) (eq 3.9)

In [ ]:
#Initialisation (Section 3.3.4)
act_len = (end-start+1) # (eq 3.41) Length of the total number of datapoints in the pulsed discharge regime
x_t = np.zeros(act_len)
xn_t = np.zeros(act_len)
xp_t = np.zeros(act_len)
xsp_t = np.zeros(act_len)
xsn_t = np.zeros(act_len)
xbp_t = np.zeros(act_len)
xbn_t = np.zeros(act_len)
qsp_t = np.zeros(act_len)
qbp_t = np.zeros(act_len)
qbn_t = np.zeros(act_len)
qsn_t = np.zeros(act_len)
qn_t = np.zeros(act_len)
qp_t = np.zeros(act_len)
V0_t = np.zeros(act_len)
Vetap_t = np.zeros(act_len)
Vetan_t = np.zeros(act_len)
Jp0_t = np.zeros(act_len)
Jn0_t = np.zeros(act_len)
VUp_t = np.zeros(act_len)
VUn_t = np.zeros(act_len)
VUpn_t = np.zeros(act_len)
VINTn_t = np.zeros(act_len)
VINTp_t = np.zeros(act_len)
Vetap_tt = np.zeros(act_len)
Vetan_tt = np.zeros(act_len)
V0_tt = np.zeros(act_len)
V_t = np.zeros(act_len)
qbsn_tt = np.zeros(act_len)
qbsp_tt = np.zeros(act_len)

In [ ]:
#Coulomb Counting (Section 3.3.4)
iapp = np.array(df1.current[start:end+1])
for i in range(act_len):
    qn_t[i] = QMAX*0.6 - np.sum(dq[start:start+i]) # (eq 3.42) At, 100% SOC, Charge in the negative terminal (qn) is 0.6 times the total capacity (0.6*QMAX). Thus, qp = 0.4*QMAX

In [ ]:
# Surface/Bulk (Positive/Negative) Distributed Charge Calculations (Section 3.2.2/3.3.4)
# Initialisation:
qp_t[0] = QMAX-qn_t[0] # eq 3.4
#Initially cs,i = cb,i
qbp_t[0] = (10/11)*qp_t[0] # eq 3.7/3.8
qsp_t[0] = (1/11)*qp_t[0] # eq 3.7/3.8
qsn_t[0] = (1/11)*qn_t[0] # eq 3.7/3.8
qbn_t[0] = (10/11)*qn_t[0] # eq 3.7/3.8
for i in range(1,act_len):
    # Euler Method with timesteps = Change in dataset's time recording (Since, it's a relatively small scale effect)
    qsp_t[i] = qsp_t[i-1] +(df1.time[start+i] - df1.time[start+i-1])*(iapp[i-1] + (1/D)*(qbp_t[i-1]/vbp - qsp_t[i-1]/vsp)) # eq 3.13
    qbp_t[i] = qbp_t[i-1] +(df1.time[start+i] - df1.time[start+i-1])*(-(1/D)*(qbp_t[i-1]/vbp - qsp_t[i-1]/vsp)) # eq 3.14
    qbn_t[i] = qbn_t[i-1] +(df1.time[start+i] - df1.time[start+i-1])*(-(1/D)*(qbn_t[i-1]/vbn - qsn_t[i-1]/vsn)) # eq 3.15
    qsn_t[i] = qsn_t[i-1] +(df1.time[start+i] - df1.time[start+i-1])*(-iapp[i-1] + (1/D)*(qbn_t[i-1]/vbn - qsn_t[i-1]/vsn)) # eq 3.16
qp_t=qsp_t+qbp_t # eq 3.10
qn_t=qsn_t+qbn_t # eq 3.10
xsn_t = (qsn_t)/((QMAX*vsn)/(vn)) # eq 3.18/3.19
xbn_t = (qbn_t)/((QMAX*vbn)/(vn)) # eq 3.18/3.19
xsp_t = (qsp_t)/((QMAX*vsp)/(vp)) # eq 3.18/3.19
xbp_t = (qbp_t)/((QMAX*vbp)/(vp)) # eq 3.18/3.19
xp_t = (qp_t)/(QMAX) # eq 3.4
xn_t = (qn_t)/(QMAX) # eq 3.4

In [ ]:
# V0,VINT and VU negative/postive electrode calculations - TimeSeries (Section 3.2.2/3.2.3/3.3.4)
for i in range (act_len):
    VINTsum = np.zeros(13)
    # eq 3.6
    for k in range (len(VINTsum)):
        VINTsum[k] = (1/(n*F))*(Ap[k])*((((2*xsp_t[i])-1)**(k+1))-(2*xsp_t[i]*k*(1-xsp_t[i])*(((2*xsp_t[i])-1)**(k-1))))
    VINTn_t[i] = (1/(n*F))*(An0)*(2*xsn_t[i]-1)  # eq 3.6
    VINTp_t[i] = np.sum(VINTsum)  # eq 3.6
    VUp_t[i] = U0p + ((R*T)/(n*F))*(math.log((1-xsp_t[i])/xsp_t[i])) + VINTp_t[i]  # eq 3.17
    VUn_t[i] = U0n + ((R*T)/(n*F))*(math.log((1-xsn_t[i])/xsn_t[i])) + VINTn_t[i]  # eq 3.17
    V0_t[i] = iapp[i]*R0 # eq 3.22

In [ ]:
# Veta_p/n calculations - TimeSeries (Section 3.2.4/3.3.4)
for i in range (act_len):
    Jp0_t[i] = kp*((1-xsp_t[i])**0.5)*((xsp_t[i])**0.5) # eq 3.25
    Jn0_t[i] = kn*((1-xsn_t[i])**0.5)*((xsn_t[i])**0.5) # eq 3.25
    Vetap_t[i] = ((R*T)/(F*0.5))*(np.arcsinh((iapp[i]/Sp)/(2*Jp0_t[i]))) # eq 3.23
    Vetan_t[i] = ((R*T)/(F*0.5))*(np.arcsinh((iapp[i]/Sn)/(2*Jn0_t[i]))) # eq 3.23

In [ ]:
# Transient Calculations - Euler Method - Timeseries (Section 3.2.6/3.3.4)
V0_tt[0] = V0_t[0] # eq 3.31
Vetap_tt[0] = Vetap_t[0] # eq 3.32
Vetan_tt[0] = Vetan_t[0] # eq 3.33
for i in range (1,act_len):
    # Euler Method with timesteps = Change in dataset's time recording (Since, it's a relatively small scale effect)
    V0_tt[i] = V0_tt[i-1] + (((df1.time[start+i] - df1.time[start+i-1])/tau0)*(V0_t[i-1]-V0_tt[i-1])) # eq 3.31
    Vetap_tt[i] = Vetap_tt[i-1] + (((df1.time[start+i] - df1.time[start+i-1])/taup)*(Vetap_t[i-1]-Vetap_tt[i-1])) # eq 3.32
    Vetan_tt[i] = Vetan_tt[i-1] + (((df1.time[start+i] - df1.time[start+i-1])/taun)*(Vetan_t[i-1]-Vetan_tt[i-1])) # eq 3.33

In [ ]:
# SOC Calculations (Section 3.2.5/3.3.4)
SOCn = np.zeros(act_len)
SOCa = np.zeros(act_len)
for i in range(act_len):
    SOCn[i] = 100*(qn_t[i]/(0.6*QMAX)) # eq 3.26
    SOCa[i] = 100*((qsn_t[i]*vn)/(0.6*QMAX*vsn)) # eq 3.27/3.28

In [ ]:
# Final Predicted Voltages (Section 3.2.6/3.3.4)
for i in range(act_len):
    V_t[i] = VUp_t[i]- VUn_t[i] - V0_tt[i] - Vetap_tt[i] - Vetan_tt[i] # eq 3.30

Plotting:
___

In [ ]:
# Dataset vs Modelled for the first pulsed discharge regime
plt.figure()
plt.axhline(y=4.2, color='k', linestyle='--', linewidth = 0.5) # 4.2 V Upper Bound (100% SOC)
plt.axhline(y=3.2, color='k', linestyle='--', linewidth = 0.5) # 3.2 V Lower Bound
plt.plot(df1.time[start:end+1]/3600 - df1.time[start]/3600,df1.voltage[start:end+1],'-k',label = "Actual",linewidth=0.75) # Dataset's Voltage (vs Time)
plt.plot(df1.time[start:end+1]/3600 - df1.time[start]/3600,V_t,'-r',label = "Modelled",linewidth=0.75) # Modelled Voltage (vs Time)
# Remove the comments on time argument to find the regime's Voltage wrt relative time
plt.ylabel('Voltage (V)', fontsize = 15)
plt.xlabel('Time (h)', fontsize = 15)
plt.title('Modelled vs Actual Voltages')
plt.legend(loc="upper right", prop={'size':10})
plt.show()